## Training 

Let's start by importing our libararies.

In [2]:
from fastai.vision import *
from fastai.vision.models import *
from tqdm import tqdm
import cv2
from fastai import *
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [3]:
labels = pd.read_csv('train.csv')
labels.Image = labels.Image.apply(lambda x: 'train/' + x)
labels.to_csv('labels.csv',index=False)

In [4]:
MODEL_PATH = 'Resnet18_v1'
TRAIN = 'train/'
TEST = 'test/'
LABELS = 'train.csv'
SAMPLE_SUB = '/sample_submission.csv'


The architecture is flexible, I chose Resnet18 since it can fit quite well into a kernel. You may play with this if you want to. 

In [5]:
arch = models.resnet18
nw = 4
LABELS

'train.csv'

Next, we prapare out dataset to work with Fastai's pipeline.

In [6]:
train_df = pd.read_csv(LABELS).set_index('Image')
unique_labels = np.unique(train_df.Id.values)

labels_dict = dict()
labels_list = []
for i in range(len(unique_labels)):
    labels_dict[unique_labels[i]] = i
    labels_list.append(unique_labels[i])
train_names = train_df.index.values
train_df.Id = train_df.Id.apply(lambda x: labels_dict[x])
train_labels = np.asarray(train_df.Id.values)

test_names = [f for f in os.listdir(TEST)]
tr_n, val_n = train_test_split(train_names, test_size=0.15, random_state=42069)
print(len(tr_n), len(val_n))

21556 3805


In [8]:
avg_width = 0
avg_height = 0
for fn in os.listdir(TRAIN)[:1000]:
    img = cv2.imread(os.path.join(TRAIN,fn))
    avg_width += img.shape[1]
    avg_height += img.shape[0]
avg_width //= 1000
avg_height //= 1000
print(avg_width, avg_height)

994 513


In [9]:
tfms = get_transforms(do_flip=False)
data = ImageDataBunch.from_csv('./', ds_tfms=tfms, size=200,bs=2)

In [10]:
learn = create_cnn(data, arch, metrics=accuracy)

In [11]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,accuracy


TypeError: Traceback (most recent call last):
  File "/home/simon/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/simon/anaconda3/lib/python3.6/site-packages/fastai/torch_core.py", line 97, in data_collate
    return torch.utils.data.dataloader.default_collate(to_data(batch))
  File "/home/simon/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 232, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/simon/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 232, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/simon/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 234, in default_collate
    raise TypeError((error_msg.format(type(batch[0]))))
TypeError: batch must contain tensors, numbers, dicts or li

Uncomment these lines to run Fastai's automatic learning rate finder. 


In [ ]:
# learn.lr_find()
# learn.sched.plot()
lr = 5e-3

We start by training only the newly initialized weights, then unfreeze the model and finetune the pretrained weights with reduced learning rate.

In [ ]:
learn.fit(lr, 1, cycle_len=1)
learn.unfreeze()
lrs = np.array([5e-4, 2.5e-4, 1.2e-4])
learn.fit(lrs, 3, cycle_len=4, use_clr=(20, 16))


In [ ]:
# sz = (avg_width, avg_height)
# batch_size = 32
# md = get_data(avg_width//2, batch_size)
# learn.set_data(md)
# learn.fit(lrs/4, 3, cycle_len=2, use_clr=(10, 8))


In [ ]:
# sz = (avg_width, avg_height)
# batch_size = 16
# md = get_data(avg_width, batch_size)
# learn.set_data(md)
# learn.fit(lrs/16, 1, cycle_len=4, use_clr=(10, 8))

## Predictions

In [ ]:
# preds_t,y_t = learn.predict_with_targs(is_test=True) # Predicting without TTA
preds_t,y_t = learn.TTA(is_test=True,n_aug=8)
preds_t = np.stack(preds_t, axis=-1)
preds_t = np.exp(preds_t)
preds_t = preds_t.mean(axis=-1)

In [ ]:
preds_t.shape

Finally, our submission.

In [ ]:
sample_df = pd.read_csv(SAMPLE_SUB)
sample_list = list(sample_df.Image)
pred_list = [[labels_list[i] for i in p.argsort()[-5:][::-1]] for p in preds_t]
pred_dic = dict((key, value) for (key, value) in zip(learn.data.test_ds.fnames,pred_list))
pred_list_cor = [' '.join(pred_dic[id]) for id in sample_list]
df = pd.DataFrame({'Image':sample_list,'Id': pred_list_cor})
df.to_csv('submission.csv'.format(MODEL_PATH), header=True, index=False)
df.head()